Demo: Diffirential Privacy trong ngữ cảnh của một database query.
* Là một cơ sở dữ liệu RẤT đơn giản với chỉ một cột boolean.
* Mỗi hàng tương ứng với một người.
* Mỗi giá trị tương ứng với việc người đó có một thuộc tính riêng nhất định hay không (chẳng hạn như họ có mắc một bệnh nào đó hay không, hoặc họ có trên / dưới một độ tuổi nhất định hay không).
* Sau đó, chúng ta sẽ tìm hiểu rằng liệu một truy vấn cơ sở dữ liệu trên một cơ sở dữ liệu nhỏ như vậy có phải là riêng tư hay không

In [ ]:
import torch
# Torch là một framework được xây dựng dựa trên python cung cấp nền tảng tính toán khoa học phục vụ lĩnh vực Deep learning
# https://pytorch.org/docs/stable/index.html

# số lượng record trong database
num_entries = 5000

db = torch.rand(num_entries) > 0.5
db

tensor([ True,  True,  True,  ..., False, False, False])

Nếu xóa ai đó khỏi cơ sở dữ liệu, kết quả đầu ra của truy vấn có 
khác không?

**Tạo các database song song**



In [ ]:
def get_parallel_db(db, removed_index):
  return torch.cat((db[0:removed_index], db[removed_index+1:]))

In [ ]:
get_parallel_db(db,3)

tensor([ True,  True,  True,  ..., False, False, False])

In [ ]:
def get_parallel_dbs(db):
  parallel_dbs = list()
  for i in range(len(db)):
    parallel_dbs.append(get_parallel_db(db, i))
  return parallel_dbs

In [ ]:
pdbs = get_parallel_dbs(db)
pdbs

In [ ]:
def create_db_and_parallels(num_entries):
  db = torch.rand(num_entries) > 0.5
  pdbs = get_parallel_dbs(db)
  
  return db, pdbs

In [ ]:
db, pdbs = create_db_and_parallels(5000)

**Tính toán quyền riêng tư của Function**

In [ ]:
def query(db):
    return db.sum()

# Độ nhạy là sự đo lường mức độ nhạy cảm của output đối với một người bị xóa khỏi cơ sở dữ liệu
# Tính độ nhạy cảm của hàm sum
full_db_result = query(db)
sensitivity = 0
for pdb in pdbs:
    pdb_result = query(pdb)
    db_distance = torch.abs(pdb_result - full_db_result)
    if(db_distance > sensitivity):
        sensitivity = db_distance

sensitivity 

tensor(1)

In [ ]:
def query_mean(db):
  return db.float().mean()
def sensitivity(query_mean, n_entries=1000):
  db, pdbs = create_db_and_parallels(n_entries)
  full_db_result = query_mean(db)
  max_distance = 0
  
  for pdb in pdbs:
    pdb_result = query_mean(pdb)
    db_distance = torch.abs(pdb_result - full_db_result)
    
    if(db_distance > max_distance):
      max_distance = db_distance
      
  return max_distance
sensitivity(query_mean)

tensor(0.0005)



*   Đối với hàm SUM, giá trị này luôn là 1. 

*   Đối với hàm MEAN, việc loại bỏ một người sẽ thay đổi kết quả của truy vấn bằng cách lấy 1 chia cho kích thước của cơ sở dữ liệu. 

=> Do đó, MEAN ít nhạy cảm hơn nhiều so với hàm SUM.

**Local Differential Privacy**

In [ ]:
def query_local_privacy(db_local):
  true_result = query_mean(db_local)

  first_coin_flip = (torch.rand(len(db_local)) > 0.5).float()
  second_coin_flip = (torch.rand(len(db_local)) > 0.5).float()
  
  db_local_noise = db_local.float() * first_coin_flip + (1-first_coin_flip)* second_coin_flip
  noise_result = query_mean(db_local_noise) * 2 - 0.5
  
  return noise_result, true_result

In [ ]:
db_local,_ = create_db_and_parallels(10)
noise_result, true_result = query_local_privacy(db_local)
print("With noise: " + str(noise_result))
print("Without noise: " + str(true_result))

With noise: tensor(0.3000)
Without noise: tensor(0.6000)


In [ ]:
db_local,_ = create_db_and_parallels(100)
noise_result, true_result = query_local_privacy(db_local)
print("With noise: " + str(noise_result))
print("Without noise: " + str(true_result))

With noise: tensor(0.4600)
Without noise: tensor(0.4400)


In [ ]:
db_local,_ = create_db_and_parallels(10000)
noise_result, true_result = query_local_privacy(db_local)
print("With noise: " + str(noise_result))
print("Without noise: " + str(true_result))

With noise: tensor(0.5086)
Without noise: tensor(0.5035)
